In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np

In [ ]:
list_symbols = pd.read_excel('C:/projeto_python/ibrx.xls')['Código'].values
list_symbols

In [ ]:
list_melhores = []
list_relacao_mm = []
list_precos = []
for ativo in list_symbols:
    soma_fundamentos = 0
    dividend_yield = 0
    chamada_api = yf.Ticker(ativo+'.SA')
    hist = chamada_api.history(period='30mo',interval = '1wk')
    hist_precos = hist['Close']
    hist_dividendos = hist['Dividends']
    hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]
    

    dividend_yield = (hist_dividendos_ultimo_ano.sum()/2)/hist_precos[-1] if hist_precos[-1] > 0 else 0
    
    if dividend_yield > 0.03:
        try:
            balanco_anual = chamada_api.balance_sheet
            balanco_trimestral = chamada_api.quarterly_balance_sheet
            dre_anual = chamada_api.financials
            dfc_anual = chamada_api.cashflow
            dre_trimestral = chamada_api.quarterly_financials
            dfc_trimestral = chamada_api.quarterly_cashflow
            
            if dre_anual.keys()[0].year >= 0:
                balanco = balanco_anual
                dre = dre_anual
                dfc = dfc_anual
                variacao_receita = int(dre.loc['Total Revenue'][0] > dre.loc['Total Revenue'][1])
                variacao_ebit = int(dre.loc['Ebit'][0] > dre.loc['Ebit'][1])
                variacao_caixa = int(dfc.loc['Change In Cash'][0] > dfc.loc['Change In Cash'][1])
                variacao_divida = int(balanco.loc['Total Current Liabilities'][0] < balanco.loc['Total Current Liabilities'][1])
                
                soma_fundamentos = variacao_caixa + variacao_ebit + variacao_receita + variacao_divida
                if soma_fundamentos >= 2:
                    print(ativo,variacao_divida,variacao_caixa,variacao_ebit,variacao_receita,dre_anual.keys()[0])
                    list_melhores.append(ativo)
                    list_relacao_mm.append(abs(hist_precos[-1] / np.mean(hist_precos[-40:])-1))
                    list_precos.append(round(hist_precos[-1],2))
            else:
                continue
        except:
            print(ativo)
            continue

In [ ]:
carteira_df = pd.DataFrame({'MM':list_relacao_mm,'Preços':list_precos}, index=list_melhores).sort_values(by = 'MM',ascending=True)

carteira_df

In [ ]:
lista_drop = ['ELET6','PETR3']

carteira_limpa = carteira_df.drop(lista_drop)


carteira_limpa

In [ ]:
list_pesos = [(1+x)/(carteira_limpa['MM'].sum()+len(carteira_limpa['MM'].values)) for x in carteira_limpa['MM'].values]
list_pesos.reverse()

list_pesos

In [ ]:
carteira_limpa['Pesos'] = list_pesos
carteira_limpa